# Data Loading and processing

In [ ]:
!git clone https://github.com/language-ml/course-nlp-ir-1-text-exploring

Cloning into 'course-nlp-ir-1-text-exploring'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 401 (delta 2), reused 0 (delta 0), pack-reused 397
Receiving objects: 100% (401/401), 98.78 MiB | 12.92 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Checking out files: 100% (245/245), done.


In [ ]:
!pip install -Uq camel_tools
import random 
import nltk
import pandas as pd
import codecs
import tqdm
import random 
import nltk
from pathlib import Path
import re
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_alef_bw
from camel_tools.utils.normalize import normalize_alef_hsb
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.utils import normalize
import _pickle as pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

quranic_dir = "/content/course-nlp-ir-1-text-exploring/exploring-datasets/religious_text"
df_quran = pd.read_csv(f'{quranic_dir}/quranic_data/id_text_with_orthographies.txt', sep='\t', header=None)
df_nahj = pd.read_csv(f'{quranic_dir}/nahj-al-balaqa/Nahj Al-Balaqa.txt', sep='\t',header=None)
verse_complete_dict = pd.Series(df_quran[1].tolist(), index=df_quran[0]).to_dict()
nahj_complete_dict = pd.Series(df_nahj[1].tolist(), index=df_nahj[0]).to_dict()

     |████████████████████████████████| 114 kB 4.3 MB/s 
     |████████████████████████████████| 4.2 MB 26.5 MB/s 
     |████████████████████████████████| 175 kB 67.6 MB/s 
     |████████████████████████████████| 418 kB 57.8 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 61.2 MB/s 
     |████████████████████████████████| 596 kB 79.6 MB/s 


In [ ]:
sahife_text=Path(f'{quranic_dir}/Saheefa/sahife_sajjadieh.txt').read_text().split('\n')
sahife_complete_dict = [re.sub('[(][۰-۹]+[)]','', item) for item in sahife_text if item.startswith('(')]

In [ ]:
sahife_complete_dict[12]

' حَمْداً یَرْتَفِعُ مِنَّا إِلَی أَعْلَی عِلِّیِّینَ فِي کِتَابٍ مَرْقُومٍ یَشْهَدُه\u200cُ الْمُقَرَّبُونَ. '

In [ ]:
nahj_complete_dict['2##186']

'وَ قَالَ ع لِلظّالِمِ الباَدیِ غَداً بِکَفّهِ عَضّةٌ'

In [ ]:
verse_complete_dict['2##186']

'وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ'

### Normalize

In [ ]:
def normalize_arabic(sentence):

    # Normalize alef variants to 'ا'
    sent_norm = normalize_unicode(sentence)
    
    sent_norm = normalize_alef_bw(sent_norm)
    # Normalize alef variants to 'ا'
    sent_norm = normalize_alef_ar(sentence)

    # Normalize alef maksura 'ى' to yeh 'ي'
    sent_norm = normalize_alef_maksura_ar(sent_norm)

    # Normalize teh marbuta 'ة' to heh 'ه'
    sent_norm = normalize_teh_marbuta_ar(sent_norm)
    return dediac_ar(sent_norm)

In [ ]:
verse_complete_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(verse_complete_dict.items())}
nahj_complete_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(nahj_complete_dict.items())}
sahife_complete_dict_nrmlz = [normalize_arabic(item) for item in tqdm.tqdm(sahife_complete_dict)]

100%|██████████| 924/924 [00:00<00:00, 9537.76it/s]


In [ ]:
print(verse_complete_dict['2##186'])
print(verse_complete_dict_nrmlz['2##186'])

print(nahj_complete_dict['2##186'])
print(nahj_complete_dict_nrmlz['2##186'])

print(sahife_complete_dict[34])
print(sahife_complete_dict_nrmlz[34])

وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ
واذا سالك عبادي عني فاني قريب ۖ اجيب دعوه الداع اذا دعان ۖ فليستجيبوا لي وليؤمنوا بي لعلهم يرشدون
وَ قَالَ ع لِلظّالِمِ الباَدیِ غَداً بِکَفّهِ عَضّةٌ
و قال ع للظالم البادی غدا بکفه عضه
 وَ کَاشَفَ فِي الدُّعَاءِ إِلَیْکَ حَامَّتَه‌ُ 
 و کاشف في الدعاء الیک حامته‌ 


In [ ]:
quranic_tokenized = [sents.split() for sents in tqdm.tqdm(verse_complete_dict_nrmlz.values())]
nahj_tokenized = [sents.split() for sents in tqdm.tqdm(nahj_complete_dict_nrmlz.values())]
sahife_tokenized = [sents.split() for sents in tqdm.tqdm(sahife_complete_dict_nrmlz)]

100%|██████████| 924/924 [00:00<00:00, 226163.45it/s]


In [ ]:
quranic_tokenized[43]

['فتلقي',
 'ادم',
 'من',
 'ربه',
 'كلمات',
 'فتاب',
 'عليه',
 'ۚ',
 'انه',
 'هو',
 'التواب',
 'الرحيم']

In [ ]:
nahj_tokenized[43]

['قبح',
 'الله',
 'مصقله',
 'فعل',
 'فعل',
 'الساده',
 'و',
 'فر',
 'فرار',
 'العبید',
 'فما',
 'انطق',
 'مادحه',
 'حتی',
 'اسکته',
 'و',
 'لا',
 'صدق',
 'واصفه',
 'حتی',
 'بکته',
 'و',
 'لو',
 'اقام',
 'لاخذنا',
 'میسوره',
 'و',
 'انتظرنا',
 'بماله',
 'وفوره']

In [ ]:
sahife_tokenized[34]

['و', 'کاشف', 'في', 'الدعاء', 'الیک', 'حامته\u200c']

# Part 1 (TF-IDF)

In [ ]:
#Normalize all verses from quran, Nahj, and Sahife
#_____________________________________________________________________________________________
verse_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(verse_complete_dict.items())}
nahj_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(nahj_complete_dict.items())}
sahife_dict_nrmlz = {normalize_arabic(v) for v in tqdm.tqdm(sahife_complete_dict)}
#_____________________________________________________________________________________________

100%|██████████| 924/924 [00:00<00:00, 7923.86it/s]


In [ ]:
# Converting standard and normalize dictionaries(index:verse) from quran, Nahj, and Sahife to Pandas DataFrame
# with column names: num and verse
#_____________________________________________________________________________________________________________
df_verse_complete_dict = pd.DataFrame(list(verse_complete_dict.items()), columns = ['num','verse'])
df_verse_complete_dict_nrmlz = pd.DataFrame(list(verse_dict_nrmlz.items()), columns = ['num','verse'])
df_nahj_complete_dict = pd.DataFrame(list(nahj_complete_dict.items()), columns = ['num','verse'])
df_nahj_complete_dict_nrmlz = pd.DataFrame(list(nahj_dict_nrmlz.items()), columns = ['num','verse'])
df_sahife_complete_dict = pd.DataFrame(list(sahife_complete_dict), columns = ['verse'])
df_sahife_complete_dict_nrmlz = pd.DataFrame(list(sahife_dict_nrmlz), columns= ['verse'])
#_____________________________________________________________________________________________________________

In [ ]:
# Create TfidfVectorizer object (first three ones are Word level and others are Character Level)
#________________________________________________________________________________________________
vectorizer_quran = TfidfVectorizer(lowercase = False)
vectorizer_nahj = TfidfVectorizer(lowercase = False)
vectorizer_sahife = TfidfVectorizer(lowercase = False)
vectorizer_quran_char = TfidfVectorizer(lowercase = False,analyzer='char')
vectorizer_nahj_char = TfidfVectorizer(lowercase = False,analyzer='char')
vectorizer_sahife_char = TfidfVectorizer(lowercase = False,analyzer='char')
#________________________________________________________________________________________________

In [ ]:
# Generate matrices of word vectors
#________________________________________________________________________________________________
tfidf_matrix_quran = vectorizer_quran.fit_transform(df_verse_complete_dict_nrmlz['verse'])
tfidf_matrix_nahj = vectorizer_nahj.fit_transform(df_nahj_complete_dict_nrmlz['verse'])
tfidf_matrix_sahife = vectorizer_sahife.fit_transform(df_sahife_complete_dict_nrmlz['verse'])
tfidf_matrix_quran_char = vectorizer_quran_char.fit_transform(df_verse_complete_dict_nrmlz['verse'])
tfidf_matrix_nahj_char = vectorizer_nahj_char.fit_transform(df_nahj_complete_dict_nrmlz['verse'])
tfidf_matrix_sahife_char = vectorizer_sahife_char.fit_transform(df_sahife_complete_dict_nrmlz['verse'])
#________________________________________________________________________________________________

# Print shapes of tfidf_matrices
#________________________________________________________________________________________________
print("Shape of tfidf_matrix(Word-Level) of quran is: ",tfidf_matrix_quran.shape)
print("Shape of tfidf_matrix(Word-Level) of Nahj is: ",tfidf_matrix_nahj.shape)
print("Shape of tfidf_matrix(Word-Level) of Sahife is: ",tfidf_matrix_sahife.shape)
print("Shape of tfidf_matrix of quran is: ",tfidf_matrix_quran_char.shape)
print("Shape of tfidf_matrix of Nahj is: ",tfidf_matrix_nahj_char.shape)
print("Shape of tfidf_matrix of Sahife is: ",tfidf_matrix_sahife_char.shape)
#________________________________________________________________________________________________

Shape of tfidf_matrix(Word-Level) of quran is:  (6236, 14658)
Shape of tfidf_matrix(Word-Level) of Nahj is:  (800, 21094)
Shape of tfidf_matrix(Word-Level) of Sahife is:  (924, 7711)
Shape of tfidf_matrix of quran is:  (6236, 40)
Shape of tfidf_matrix of Nahj is:  (800, 36)
Shape of tfidf_matrix of Sahife is:  (924, 47)


In [ ]:
#extract all (index:verse) pairs in DataFrames
#____________________________________________________________________________________________________________________
quran_indices_nrmlz = pd.Series(df_verse_complete_dict_nrmlz.index, index = df_verse_complete_dict_nrmlz['verse'])
nahj_indices_nrmlz = pd.Series(df_nahj_complete_dict_nrmlz.index, index = df_nahj_complete_dict_nrmlz['verse'])
sahife_indices_nrmlz = pd.Series(df_sahife_complete_dict_nrmlz.index, index = df_sahife_complete_dict_nrmlz['verse'])
#____________________________________________________________________________________________________________________

In [ ]:
# Here you should enter input sentence for TF-IDF based similarity detection
#____________________________________________________________________________________________________________________

input = [normalize_arabic('مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ')]
#____________________________________________________________________________________________________________________

In [ ]:
def get_similars_from_quran(input, vectors, indices, mode='word'):
    """
    Function for find input's similar verses from quran
    input: input sentence
    vectors: TF-IDF matrix
    indices: (index:verse) pairs from quran
    mode: 'word' for Word-Level or 'char' for Character-Level
    """

    if mode == 'word':
          tfidf_matrix_input = vectorizer_quran.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_quran)
    elif mode == 'char':
          tfidf_matrix_input = vectorizer_quran_char.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_quran_char)

    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(linear_sim[0]))

    # Sort the verses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar verses
    sim_scores = sim_scores[0:10]

    # Get the verses indices
    indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar verses
    return df_verse_complete_dict['verse'].iloc[indices]

In [ ]:
def get_similars_from_nahj(input, vectors, indices, mode = 'word'):
    """
    Function for find input's similar verses from Nahj
    input: input sentence
    vectors: TF-IDF matrix
    indices: (index:verse) pairs from quran
    mode: 'word' for Word-Level or 'char' for Character-Level
    """
    if mode == 'word':
          tfidf_matrix_input = vectorizer_nahj.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_nahj)
    elif mode == 'char':
          tfidf_matrix_input = vectorizer_nahj_char.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_nahj_char)


    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(linear_sim[0]))

    # Sort the verses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar verses
    sim_scores = sim_scores[0:10]

    # Get the verses indices
    indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar verses
    return df_nahj_complete_dict['verse'].iloc[indices]

In [ ]:
def get_similars_from_sahife(input, vectors, indices, mode='word'):
    """
    Function for find input's similar verses from Sahife
    input: input sentence
    vectors: TF-IDF matrix
    indices: (index:verse) pairs from quran
    mode: 'word' for Word-Level or 'char' for Character-Level
    """
    if mode == 'word':
          tfidf_matrix_input = vectorizer_sahife.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_sahife)
    elif mode == 'char':
          tfidf_matrix_input = vectorizer_sahife_char.transform(input)
          linear_sim = linear_kernel(tfidf_matrix_input , tfidf_matrix_sahife_char)

    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(linear_sim[0]))

    # Sort the verses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar verses
    sim_scores = sim_scores[0:10]

    # Get the verses indices
    indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar verses
    return df_sahife_complete_dict['verse'].iloc[indices]

In [ ]:
# Find similar verses
#____________________________________________________________________________________________________________________
QW = pd.DataFrame(get_similars_from_quran(input, vectorizer_quran , quran_indices_nrmlz, 'word'))
NW = pd.DataFrame(get_similars_from_nahj(input, vectorizer_nahj , nahj_indices_nrmlz, 'word'))
SW = pd.DataFrame(get_similars_from_sahife(input, vectorizer_sahife , sahife_indices_nrmlz, 'word'))
#____________________________________________________________________________________________________________________
print("Similar verses from quran (WORD-LEVEL):")
print(QW.to_markdown())
print("Similar verses from Nahj (WORD-LEVEL):")
print(NW.to_markdown())
print("Similar verses from Sahife (WORD-LEVEL):")
print(SW.to_markdown())

Similar verses from quran (WORD-LEVEL):
|      | verse                                                                              |
|-----:|:-----------------------------------------------------------------------------------|
| 6233 | مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ                                                               |
| 6226 | مِنْ شَرِّ مَا خَلَقَ                                                                       |
| 6228 | وَمِنْ شَرِّ النَّفَّاثَاتِ فِي الْعُقَدِ                                                           |
| 6227 | وَمِنْ شَرِّ غَاسِقٍ إِذَا وَقَبَ                                                                |
| 6229 | وَمِنْ شَرِّ حَاسِدٍ إِذَا حَسَدَ                                                                |
| 1214 | إِنَّ شَرَّ الدَّوَابِّ عِنْدَ اللَّهِ الَّذِينَ كَفَرُوا فَهُمْ لَا يُؤْمِنُونَ                                    |
| 1181 | إِنَّ شَرَّ الدَّوَابِّ عِنْدَ اللَّهِ الصُّمُّ الْبُكْمُ الَّذِينَ لَا يَعْقِلُونَ              

In [ ]:
# Shuffle Quran Word-Level similar verses for MRR calculation
QW_shuffled = QW.sample(frac=1)
print(QW_shuffled.to_markdown())

|      | verse                                                                              |
|-----:|:-----------------------------------------------------------------------------------|
| 6233 | مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ                                                               |
| 1181 | إِنَّ شَرَّ الدَّوَابِّ عِنْدَ اللَّهِ الصُّمُّ الْبُكْمُ الَّذِينَ لَا يَعْقِلُونَ                                   |
| 6227 | وَمِنْ شَرِّ غَاسِقٍ إِذَا وَقَبَ                                                                |
| 6135 | إِنَّ الَّذِينَ كَفَرُوا مِنْ أَهْلِ الْكِتَابِ وَالْمُشْرِكِينَ فِي نَارِ جَهَنَّمَ خَالِدِينَ فِيهَا ۚ أُولَٰئِكَ هُمْ شَرُّ الْبَرِيَّةِ |
| 2888 | الَّذِينَ يُحْشَرُونَ عَلَىٰ وُجُوهِهِمْ إِلَىٰ جَهَنَّمَ أُولَٰئِكَ شَرٌّ مَكَانًا وَأَضَلُّ سَبِيلًا                         |
| 5601 | فَوَقَاهُمُ اللَّهُ شَرَّ ذَٰلِكَ الْيَوْمِ وَلَقَّاهُمْ نَضْرَةً وَسُرُورًا                                        |
| 1214 | إِنَّ شَرَّ الدَّوَابِّ عِنْدَ اللَّهِ ال

In [ ]:
# Shuffle Nahj Word-Level similar verses for MRR calculation
NW_shuffled = NW.sample(frac=1)
print(NW_shuffled.to_markdown())

|     | verse                                                                                                                                                                                                                                                                                                                                                              |
|----:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 245 | وَ إِنّ عَمَلَکَ لَیسَ لَکَ بِطُعمَةٍ وَ لَکِنّهُ فِی عُنُقِکَ أَمَانَةٌ وَ أَنتَ مُستَرعًی لِمَن فَوقَکَ لَیسَ لَکَ أَن تَفتَاتَ فِی رَعِیّةٍ وَ لَا تُخَاطِرَ إِلّا بِوَثِیقَةٍ وَ فِی یَدَیکَ مَالٌ مِن مَالِ اللّهِ عَزّ وَ جَلّ وَ أَنتَ مِن خُزّانِهِ حَتّی تُسَ

In [ ]:
# Shuffle Sahife Word-Level similar verses for MRR calculation
SW_shuffled = SW.sample(frac=1)
print(SW_shuffled.to_markdown())

|     | verse                                                                                                                                                                                                                                                                                                                                     |
|----:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 153 | وَ نَعُوذُ بِکَ أَنْ نَنْطَوِيَ عَلَی غِشِّ أَحَدٍ، وَ أَنْ نُعْجِبَ بِأَعْمَالِنَا، وَ نَمُدَّ فِي آمَالِنَا                                                                                                                                                                                                           

In [ ]:
# Find similar verses
#____________________________________________________________________________________________________________________
QC = pd.DataFrame(get_similars_from_quran(input, vectorizer_quran , quran_indices_nrmlz, 'char'))
NC = pd.DataFrame(get_similars_from_nahj(input, vectorizer_nahj , nahj_indices_nrmlz, 'char'))
SC = pd.DataFrame(get_similars_from_sahife(input, vectorizer_sahife , sahife_indices_nrmlz, 'char'))
#____________________________________________________________________________________________________________________

print("Similar verses from quran (CHAR-LEVEL):")
print(QC.to_markdown())
print("Similar verses from Nahj (CHAR-LEVEL):")
print(NC.to_markdown())
print("Similar verses from Sahife (CHAR-LEVEL):")
print(SC.to_markdown())

Similar verses from quran (CHAR-LEVEL):
|      | verse                                                                         |
|-----:|:------------------------------------------------------------------------------|
| 6233 | مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ                                                          |
| 4266 | لَا يَسْأَمُ الْإِنْسَانُ مِنْ دُعَاءِ الْخَيْرِ وَإِنْ مَسَّهُ الشَّرُّ فَيَئُوسٌ قَنُوطٌ                         |
| 6177 | إِنَّ الْإِنْسَانَ لَفِي خُسْرٍ                                                            |
| 4822 | وَأَنْ لَيْسَ لِلْإِنْسَانِ إِلَّا مَا سَعَىٰ                                                    |
| 5489 | إِنَّا أَرْسَلْنَا إِلَيْكُمْ رَسُولًا شَاهِدًا عَلَيْكُمْ كَمَا أَرْسَلْنَا إِلَىٰ فِرْعَوْنَ رَسُولًا                 |
| 5814 | فَلَا أُقْسِمُ بِالْخُنَّسِ                                                               |
| 3114 | وَلَا تَبْخَسُوا النَّاسَ أَشْيَاءَهُمْ وَلَا تَعْثَوْا فِي الْأَرْضِ مُفْسِدِينَ                    

In [ ]:
# Shuffle Quran Char-Level similar verses for MRR calculation
QC_shuffled = QC.sample(frac=1)
print(QC_shuffled.to_markdown())

|      | verse                                                                         |
|-----:|:------------------------------------------------------------------------------|
| 3400 | وَلَئِنْ سَأَلْتَهُمْ مَنْ خَلَقَ السَّمَاوَاتِ وَالْأَرْضَ وَسَخَّرَ الشَّمْسَ وَالْقَمَرَ لَيَقُولُنَّ اللَّهُ ۖ فَأَنَّىٰ يُؤْفَكُونَ |
| 5489 | إِنَّا أَرْسَلْنَا إِلَيْكُمْ رَسُولًا شَاهِدًا عَلَيْكُمْ كَمَا أَرْسَلْنَا إِلَىٰ فِرْعَوْنَ رَسُولًا                 |
| 3114 | وَلَا تَبْخَسُوا النَّاسَ أَشْيَاءَهُمْ وَلَا تَعْثَوْا فِي الْأَرْضِ مُفْسِدِينَ                            |
| 6233 | مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ                                                          |
| 5814 | فَلَا أُقْسِمُ بِالْخُنَّسِ                                                               |
| 1782 | وَسَخَّرَ لَكُمُ الشَّمْسَ وَالْقَمَرَ دَائِبَيْنِ ۖ وَسَخَّرَ لَكُمُ اللَّيْلَ وَالنَّهَارَ                          |
| 4822 | وَأَنْ لَيْسَ لِلْإِنْسَانِ إِلَّا مَا سَعَىٰ                                   

In [ ]:
# Shuffle Nahj Char-Level similar verses for MRR calculation
NC_shuffled = NC.sample(frac=1)
print(NC_shuffled.to_markdown())

|     | verse                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|----:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Shuffle Sahife Char-Level similar verses for MRR calculation
SC_shuffled = SC.sample(frac=1)
print(SC_shuffled.to_markdown())

|     | verse                                                                                                                                                                                                                                                                                                         |
|----:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 273 | اللَّهُمَّ صَلِّ عَلَی مُحَمَّدٍ وَ آلِه‌ِ، وَ قِنِي مِنَ الْمَعَاصِي، وَ اسْتَعْمِلْنِي بِالطَّاعَةِ، وَ ارْزُقْنِي حُسْنَ الْإِنَابَةِ، وَ طَهِّرْنِي بِالتَّوْبَةِ، وَ أَیِّدْنِي بِالْعِصْمَةِ، وَ اسْتَصْلِحْنِي بِالْعَافِیَةِ، وَ أَذِقْنِي حَلَاوَةَ الْمَغْفِرَةِ، وَ اجْعَلْنِي طَلِیقَ عَفْوِکَ، وَ عَتِیقَ رَحْمَتِکَ، وَ اکْتُبْ لِي أَمَاناً مِنْ سُخْطِکَ، وَ بَش